In [90]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
# Echarts Jupyter lab support
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

In [84]:
# Echarts imports
from pyecharts import options as opts
from pyecharts.globals import  ThemeType
from pyecharts.charts import Line

In [85]:
from IPython.display import HTML
import matplotlib.pyplot as plt

In [86]:
from common.log_helper import LogHelper
from backtest.scale_backtest_args import ScaleBacktestArgGen
from backtest.scale_chain_reader import ScaleChainReader
from notebooks.img_symbols import down_arrow
from notebooks.echart_df_plot import plot_df_simple
from md.instrument_manager import InstrumentType
from md.instrument_manager import InstrumentManager, InstrumentType, InstField
from modelservice.supp_res import OptSuppRes, Summary

from datetime import datetime
import logging
import sys
import pandas as pd


In [6]:
LogHelper.configure_logging(verbose=False)
pd.set_option("display.precision", 2)
pd.set_option('display.float_format', '{:.2f}'.format)

In [8]:
ReaderArgs = {
    'common': {
        'start_time': '09:30:00',
        'end_time' : '09:30:00',
    },
}

In [9]:
dump_base_dir = "/home/qoptrprod/data/dumps"
trading_date = "20240320"
underlying = "NIFTY_I"

In [10]:
arg_obj = ScaleBacktestArgGen.resolve_and_gen_args(dump_base_dir, underlying, trading_date, ReaderArgs)
reader = ScaleChainReader(arg_obj)

In [11]:
analysis_recs = []
for chain in reader.read_chain():
    print(type(chain.get_ul_exch_ts()))
    analysis_recs.append({
        'ts' : chain.get_ul_exch_ts(),
        'ulsy': chain.get_ul_symbol(),
        'futsy' : chain.get_trd_symbol(), 
        'optsy' : chain.get_opt_symbol(),
        'trdinstid' : chain.trd_instid,
        'lot_size' : chain.get_trd_lot_size(), 
        'ulpx' : chain.get_ul_price(),
        'futpx' : chain.get_trd_ltp(),
        'kgap' : chain.get_opt_strike_gap(),
        'mdatm' : chain.get_md_atm(),
        'mda_cltp' : chain.get_opt_ltp(strike=chain.get_md_atm(), instr_type=InstrumentType.CE),
        'mda_pltp' : chain.get_opt_ltp(strike=chain.get_md_atm(), instr_type=InstrumentType.PE),
    })
    break

<class 'str'>


In [12]:
print(analysis_recs[0])

{'ts': '2024-03-20 09:30:00', 'ulsy': 'NIFTY', 'futsy': 'NIFTY24MARFUT', 'optsy': 'NIFTY', 'trdinstid': 9372674, 'lot_size': 50, 'ulpx': 21855.3, 'futpx': 21934.95, 'kgap': 50.0, 'mdatm': 21850.0, 'mda_cltp': 110.6, 'mda_pltp': 85.05}


In [13]:
df = pd.DataFrame(analysis_recs)
_ = df.set_index('ts')

In [14]:
df.head(5)

,ts,ulsy,futsy,optsy,trdinstid,lot_size,ulpx,futpx,kgap,mdatm,mda_cltp,mda_pltp
0,2024-03-20 09:30:00,NIFTY,NIFTY24MARFUT,NIFTY,9372674,50,21855.30,21934.95,50.00,21850.00,110.60,85.05


In [47]:
cdf = chain.get_opt_dataframe()    

In [49]:
cdf.head(2)

,strike,c_px,p_px,c_vol,p_vol,c_oi,p_oi
0,21100.00,769.95,0.90,500,2088300,2200,2230300
1,21150.00,675.30,1.00,0,667300,450,972200


In [87]:
sr = OptSuppRes(chain)

In [102]:
cs, ps = sr.process()

In [103]:
cs, ps

(Summary(leader='VOL', state='STG', strike=21900.0, h2k=0.0),
 Summary(leader='VOL', state='XDW', strike=21900.0, h2k=21800.0))